In [1]:
import tensorflow as tf
import tensorflow.keras as K

In [2]:
def upscale_layer(layer, upscale_factor):
  '''
  upscale_factor 만큼 층(tensor)을 업스케일
  tensor 크기는 [group, height, weight, channels]
  '''
  height, width = layer.get_shape()[1:3]
  size = (upscale_factor * height, upscale_factor * width)
  upscaled_layer = tf.image.resize_nearest_neighbor(layer, size) #최근접 이웃 보간을 사용하여 resize
  return upscaled_layer

In [3]:
def smoothly_merge_last_layer(list_of_layers, alpha):
  '''
  임곗값 알파를 기반으로 층을 부드럽게 합침
  모든 층이 RGB로 바뀌었다고 가정
  생성자를 위한 함수
  list_of_layers : 해상도 순서대로 정렬된 텐서 리스트
  alpha : 0~1 사이의 실수
  '''

  last_fully_trained_layer = list_of_layers[-2]
  # 업스케일링을 위해 끝에서 두 번째 층 선택
  last_layer_upscaled = upscale_layer(last_fully_trained_layer, 2)
  # 마지막으로 훈련된 층을 업스케일링
  larger_native_layer = list_of_layers[-1]
  # 새로 추가된 층은 아직 완전히 훈련되지 않음

  assert larger_native_layer.get_shape() == last_layer_upscaled.get_shape()
  #합치기 전 층 크기가 같은지 확인
  new_layer = (1-alpha) * last_layer_upscaled + alpha * larger_native_layer
  # 곱셈은 브로드캐스팅되어 수행
  return new_layer